In [1]:
!pip install torch

In [2]:
from __future__ import unicode_literals, print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import re
import random

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
torch.cuda.get_device_name()

'Tesla T4'

In [4]:
sos_token = 0 # start of sequence token
eos_token = 1 # end of sequence token
max_length = 20

class Lang:
    def __init__(self):
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2 # Counting SOS and EOS token

    # split sentence to word and add to container
    def add_sentence(self, sentence):
        for word in sentence.split(" "):
            self.add_word(word)

    # if word is not in container, it will be added to it else we update word counter
    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] +=1

In [5]:
def normalize_sentence1(df, lang1):
    sentence = df[lang1].str.lower()
    sentence = sentence.str.replace('[^A-Za-z\s]+', "")
    sentence = sentence.str.normalize('NFD')
    sentence = sentence.str.encode("ascii", errors="ignore").str.decode("utf-8")
    return sentence

def normalize_sentence2(df, lang2):
    sentence = df[lang2].str.lower()
    sentence = sentence.str.replace('[^ぁ-んァ-ン一-龯ー 　]+', "")
    sentence = sentence.str.normalize('NFKC')
    return sentence

def read_sentence(df, lang1, lang2):
    sentence1 = normalize_sentence1(df, lang1)
    sentence2 = normalize_sentence2(df, lang2)
    return sentence1, sentence2

def read_file(loc, lang1, lang2):
    df = pd.read_csv(loc, delimiter="\t", header=None, names=[lang1, lang2, "extra"])
    df.drop(["extra"], inplace=True, axis=1)
    return df

def process_data(lang1, lang2):
    df = read_file(f'/content/{lang1}-{lang2}.txt', lang1, lang2)
    print(f"Read {len(df)} Sentence Pairs")
    sentence1, sentence2 = read_sentence(df, lang1, lang2)
    source = Lang()
    target = Lang()

    pairs = []
    for i in range(len(df)):
        if len(sentence1[i].split(" ")) < max_length and len(sentence2[i].split(" ")) < max_length:
            full = [sentence1[i], sentence2[i]]
            source.add_sentence(sentence1[i])
            target.add_sentence(sentence2[i])
            pairs.append(full)

    return source, target, pairs

In [6]:
def indexes_from_sentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(" ")]

def tensor_from_sentence(lang, sentence):
    indexes = indexes_from_sentence(lang, sentence)
    indexes.append(eos_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensors_from_pair(input_lang, output_lang, pair):
    input_tensor = tensor_from_sentence(input_lang, pair[0])
    target_tensor = tensor_from_sentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [7]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, embedded_dim, num_layers):
        super(Encoder, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.embedded_dim = embedded_dim
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_dim, self.embedded_dim)
        self.gru = nn.GRU(self.embedded_dim, self.hidden_dim, num_layers = self.num_layers)

    def forward(self, src):
        embedded = self.embedding(src). view(1, 1, -1)
        outputs, hidden = self.gru(embedded)
        return outputs, hidden

In [8]:
class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, embedded_dim, num_layers):
        super(Decoder, self).__init__()
        self.embedded_dim = embedded_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.embedding = nn.Embedding(self.output_dim, self.embedded_dim)
        self.gru = nn.GRU(self.embedded_dim, self.hidden_dim, num_layers=self.num_layers)
        self.out = nn.Linear(self.hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        input = input.view(-1, 1)
        embedded = F.relu(self.embedding(input))
        output, hidden = self.gru(embedded, hidden)
        prediction = self.softmax(self.out(output[0]))
        return prediction, hidden

In [9]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device, max_length=max_length):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, source, target, teacher_forcing_ratio=0.5):
        input_length = source.size(0)
        batch_size = target.shape[1]
        target_length = target.shape[0]
        vocab_size = self.decoder.output_dim
        outputs = torch.zeros(target_length, batch_size, vocab_size).to(self.device)

        for i in range(input_length):
            _, encounter_hidden = self.encoder(source[i])
        decoder_hidden = encounter_hidden.to(device)
        decoder_input = torch.tensor([sos_token], device=device)
        for t in range(target_length):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            outputs[t] = decoder_output
            teacher_force = random.random() < teacher_forcing_ratio
            _, topi = decoder_output.topk(1)
            input = (target[t] if teacher_force else topi)
            if (teacher_force == False and input.item() == eos_token):
                break

        return outputs

In [10]:
teacher_forcing_ratio = 0.5

def clacModel(model, input_tensor, target_tensor, model_optimizer, criterion):
    model_optimizer.zero_grad()
    # input_length = input_tensor.size(0)
    loss = 0
    epoch_loss = 0

    output = model(input_tensor, target_tensor)

    num_iter = output.size(0)

    for ot in range(num_iter):
        loss += criterion(output[ot], target_tensor[ot])

    loss.backward()
    model_optimizer.step()
    epoch_loss = loss.item()/num_iter

    return epoch_loss

In [11]:
def train_model(model, source, target, pairs, num_iteration=20000):
    model.train()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    criterion = nn.NLLLoss() # best used in conjunction with log softmax activation, i.e., negative log likelihood loss
    total_loss_iterations = 0

    training_pairs = [tensors_from_pair(source, target, random.choice(pairs)) for i in range(num_iteration)]

    for iter in tqdm(range(1, num_iteration+1)):
        training_pair = training_pairs[iter-1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = clacModel(model, input_tensor, target_tensor, optimizer, criterion)

        total_loss_iterations += loss

        if iter%5000 == 0:
            average_loss = total_loss_iterations/5000
            total_loss_iterations = 0
            print(f"{iter} - {average_loss:.4f}")

    torch.save(model.state_dict(), 'seq2seq.pt')

    return model

In [12]:
def evaluate(model, input_lang, output_lang, sentences, max_length=max_length):
    with torch.no_grad():
        input_tensor = tensor_from_sentence(input_lang, sentences[0])
        output_tensor = tensor_from_sentence(output_lang, sentences[1])

        decoded_words = []

        output = model(input_tensor, output_tensor)
        for ot in range(output.size(0)):
            _, topi = output[ot].topk(1)
            print(topi)
            if topi[0].item() == eos_token:
                decoded_words.append("<EOS>")
                break
            else:
                decoded_words.append(output_lang.index2word[topi[0].item()])

    return decoded_words

def evaluate_randomly(model, source, target, pairs, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print(f"Source: {pair[0]}")
        print(f"Target: {pair[1]}")

        output_words = evaluate(model, source, target, pair)

        output_sentence = " ".join(output_words)

        print(f"Predicted: {output_sentence}")

In [13]:
lang1 = "eng"
lang2 = "jp"

source, target, pairs = process_data(lang1, lang2)

Read 104785 Sentence Pairs


<ipython-input-5-adea5dfb4eb1>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  sentence = sentence.str.replace('[^A-Za-z\s]+', "")
<ipython-input-5-adea5dfb4eb1>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  sentence = sentence.str.replace('[^ぁ-んァ-ン一-龯ー 　]+', "")


In [14]:
source.__dict__["word2index"]["hi"], source.__dict__["n_words"] # english words

(3, 11664)

In [15]:
target.__dict__["index2word"][11], target.__dict__["n_words"] # japanese words

('ワォ', 86712)

In [16]:
pairs[:10]

[['go', '行け'],
 ['go', '行きなさい'],
 ['hi', 'こんにちは'],
 ['hi', 'もしもし'],
 ['hi', 'やっほー'],
 ['hi', 'こんにちは'],
 ['run', '走れ'],
 ['run', '走って'],
 ['who', '誰'],
 ['wow', 'すごい']]

In [17]:
randomize = random.choice(pairs)
print(f"Random Sentence: {randomize}")

Random Sentence: ['i have but one wish', '僕のお願いは一つだけだよ']


In [18]:
input_size = source.n_words
output_size = target.n_words

print(f"Input: {input_size}, Output: {output_size}")

Input: 11664, Output: 86712


In [19]:
embed_size = 256
hidden_size = 512
num_layers = 1
num_iteration = 1000000

encoder = Encoder(input_size, hidden_size, embed_size, num_layers)
decoder = Decoder(output_size, hidden_size, embed_size, num_layers)

model = Seq2Seq(encoder, decoder, device).to(device)

print(encoder)
print(decoder)

Encoder(
  (embedding): Embedding(11664, 256)
  (gru): GRU(256, 512)
)
Decoder(
  (embedding): Embedding(86712, 256)
  (gru): GRU(256, 512)
  (out): Linear(in_features=512, out_features=86712, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [20]:
model = train_model(model, source, target, pairs, num_iteration)

  1%|          | 5009/1000000 [01:09<3:56:56, 69.99it/s]

5000 - 5.7097


  1%|          | 10008/1000000 [02:17<3:44:32, 73.48it/s]

10000 - 5.6990


  2%|▏         | 15013/1000000 [03:25<3:16:27, 83.56it/s]

...

990000 - 5.6525


100%|█████████▉| 995012/1000000 [4:15:07<01:22, 60.60it/s]

995000 - 5.6597


100%|██████████| 1000000/1000000 [4:16:31<00:00, 64.97it/s]


1000000 - 5.6566


In [21]:
evaluate_randomly(model, source, target, pairs)

Source: i like taking pictures
Target: 写真を撮るのが好きです
tensor([[62]], device='cuda:0')
tensor([[1]], device='cuda:0')
Predicted: 落ち着いて <EOS>
Source: its impossible for me to finish my term paper by tomorrow
Target: 明日までに期末レポートを仕上げるなんて不可能だよ
tensor([[62]], device='cuda:0')
tensor([[1]], device='cuda:0')
Predicted: 落ち着いて <EOS>
Source: tom lives here alone
Target: トムはここに独りで住んでいる
tensor([[62]], device='cuda:0')
tensor([[1]], device='cuda:0')
Predicted: 落ち着いて <EOS>
Source: please put this box away for me
Target: この箱を片付けておいてくれ
tensor([[62]], device='cuda:0')
tensor([[1]], device='cuda:0')
Predicted: 落ち着いて <EOS>
Source: i woke up in the hospital
Target: 目が覚めると病院だった
tensor([[62]], device='cuda:0')
tensor([[1]], device='cuda:0')
Predicted: 落ち着いて <EOS>
Source: we heard a knock on the door
Target: ドアをノックする音が聞こえました
tensor([[62]], device='cuda:0')
tensor([[1]], device='cuda:0')
Predicted: 落ち着いて <EOS>
Source: im too busy to go
Target: 私は忙しくて行けない
tensor([[62]], device='cuda:0')
tensor([[1]], device='cuda: